<a href="https://colab.research.google.com/github/Habibur-02/Generative-Adversarial-Networks-GANs/blob/main/DCGANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# DCGAN= Deep Convolution GANs
Upsampling convolution
It is also called often.
`Deconvolution or
Fractionally strided convolution or
Upconvolution`

# Output Size= (I-1)*S-2P+K
I=Input size
S=Stride
P=Padding
K=Kernel


In [ ]:
import torch
import torch.nn as nn

input=torch.randn(1,1,6,6)
input

model=nn.Sequential(
    nn.Conv2d(in_channels=1,
             out_channels=10,
             kernel_size=2,
             padding=1,
             stride=1,
             bias=True),
    nn.LeakyReLU(0.3),
    nn.BatchNorm2d(10),
    nn.Dropout(0.3),
    nn.Tanh()

)

output=model(input)
output.shape
# output

In [ ]:
import torch
import torch.nn as nn

input= torch.randn(1,3,8,8)
input

model=nn.Sequential(
    nn.ConvTranspose2d(
        in_channels=3,
        out_channels=1,
        kernel_size=2,
             padding=1,
             stride=1,
             bias=True

    ),
    nn.LeakyReLU(0.3),
    nn.BatchNorm2d(1),
    nn.Dropout(0.3),
    nn.Tanh()
)

output=model(input)
output.shape

output


# f(x)=x if x≥0
#      αx if x<0

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import os

latent_dim = 100
img_size = 32
channels = 3
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 1
lr = 0.0002
beta1 = 0.5

os.makedirs("dcgan_cifar10_output", exist_ok=True)

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Pixel: [-1, 1]
])

dataloader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10(root="./data", download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)



In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

latent_dim = 100
img_channels = 3
img_size = 32
batch_size = 128
lr = 0.0002
beta1 = 0.5
epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs("dcgan_outputs", exist_ok=True)

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model=nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0), #in_chan,out_chan, kernel, stride, padd
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.ConvTranspose2d(512, 256, 4, 2,1), #in_chan,out_chan, kernel, stride, padd
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),       # (B,128,16,16)
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, img_channels, 4, 2, 1),# (B,3,32,32)
            nn.Tanh()
        )
    def forward(self,z):
        return self.model(z)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model= nn.Sequential(
            nn.Conv2d(img_channels, 128, 4,2,1), #[B,128, 16, 16]
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128,256,4,2,1),   #8*8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1),  #4*4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512,1,4,1,0),
            nn.Sigmoid()


        )
    def forward(self, img):
        out=self.model(img)
        return out.view(-1,1)


model_Gen= Generator().to(device)
model_Gen

model_Dis=Discriminator().to(device)

model_Dis


In [ ]:
loss_fn=nn.BCELoss()

optimizer_G=torch.optim.Adam(params=model_Gen.parameters(),lr=lr, betas=(beta1, 0.999))
optimizer_D=torch.optim.Adam(params=model_Dis.parameters(),lr=lr, betas=(beta1, 0.999))


fixed_noise=torch.randn(64, latent_dim, 1, 1, device=device)




In [ ]:
for i, (imgs, _) in enumerate(train_loader):
    if i>1:
        break
    imgs=imgs.to(device)
    batch=imgs.size(0)
    real_label=torch.full((batch, 1), 0.9, device=device)
    fake_label=torch.zeros(batch, 1).to(device)
    # real_out=model_Dis()
    print(imgs.shape)
    #trainging discriminator
    optimizer_D.zero_grad()
    z=torch.randn(batch, latent_dim, 1, 1, device=device)
    real_out=model_Dis(imgs)
    real_loss=loss_fn(real_out, real_label)

    fake_imgs=model_Gen(z)
    fake_out=model_Dis(fake_imgs.detach())
    fake_loss=loss_fn(fake_out, fake_label)

    total_loss=real_loss+fake_loss
    print(total_loss.item())
    total_loss.backward()
    optimizer_D.step()
    #Generator training
    optimizer_G.zero_grad()


    gen_img=model_Gen(z)
    dis_out_val=model_Dis(gen_img)

    gen_loss=loss_fn(dis_out_val,real_label)
    print(gen_loss.item())
    gen_loss.backward()
    optimizer_G.step()


    if i%1==0:
        print(f"i->{i} D[{total_loss.item()}] G[{gen_loss.item()}]")

    with torch.no_grad():
        generated_imgs=model_Gen(fixed_noise).detach().cpu()
        save_image(fake_sample, f"dcgan_outputs/fake_{epoch:03d}_{k:04d}.png", normalize=True)



torch.save(model_Gen.state_dict(),"dcgan_generate.pth")
torch.save(model_Dis.state_dict(),"dcgan_discriminator.pth")




In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

# ========== Hyperparameters ==========
latent_dim = 100
img_channels = 3
img_size = 32
batch_size = 128
lr = 0.0002
beta1 = 0.5
epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs("dcgan_outputs", exist_ok=True)

# ========== DataLoader ==========
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# ========== Generator ==========
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, img_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# ========== Discriminator ==========
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(img_channels, 128, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0),
            nn.Sigmoid()
        )

    def forward(self, img):
        out = self.model(img)
        return out.view(-1, 1)

# ========== Model Initialization ==========
model_G = Generator().to(device)
model_D = Discriminator().to(device)

# ========== Loss & Optimizers ==========
loss_fn = nn.BCELoss()
optimizer_G = torch.optim.Adam(model_G.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = torch.optim.Adam(model_D.parameters(), lr=lr, betas=(beta1, 0.999))

# ========== Fixed Noise ==========
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

# ========== Training Loop ==========
k = 0
for epoch in range(epochs):
    for i, (real_imgs, _) in enumerate(train_loader):
        if i>0:
            break
        current_batch = real_imgs.size(0)
        real_imgs = real_imgs.to(device)

        # Real and fake labels
        real_label = torch.full((current_batch, 1), 0.9, device=device)
        fake_label = torch.zeros(current_batch, 1, device=device)

        # ======== Train Discriminator ========
        optimizer_D.zero_grad()

        real_out = model_D(real_imgs)
        real_loss = loss_fn(real_out, real_label)

        noise = torch.randn(current_batch, latent_dim, 1, 1, device=device)
        fake_imgs = model_G(noise)
        fake_out = model_D(fake_imgs.detach())
        fake_loss = loss_fn(fake_out, fake_label)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # ======== Train Generator ========
        optimizer_G.zero_grad()
        output = model_D(fake_imgs)
        g_loss = loss_fn(output, real_label)
        g_loss.backward()
        optimizer_G.step()

        # ======== Logging (optional but helpful) ========
        if i % 100 == 0:
            print(f"[Epoch {epoch+1}/{epochs}] [Batch {i}/{len(train_loader)}] "
                  f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

        # ======== Save Sample Images ========
        if i % 500 == 0:
            with torch.no_grad():
                fake_sample = model_G(fixed_noise).detach().cpu()
                save_image(fake_sample, f"dcgan_outputs/fake_{epoch:03d}_{k:04d}.png", normalize=True)
            k += 1

# ========== Save Final Model ==========
torch.save(model_G.state_dict(), "dcgan_generator.pth")
torch.save(model_D.state_dict(), "dcgan_discriminator.pth")
